In [1]:
import cv2
import numpy as np

import ipywidgets as widgets
from IPython.display import display, clear_output
import time

from tiki.mini import TikiMini

In [2]:
tiki = TikiMini()

In [3]:
# Hyper Parameters
# 실전용
#lower_event = np.array([67, 23, 101])
#upper_event = np.array([80, 255, 168])

# 연습용
lower_event = np.array([40, 70, 50])
upper_event = np.array([80, 150, 150])

# Byung Ddu ggung
#lower_event = np.array([90, 100, 100])
#upper_event = np.array([150, 255, 255])

# 절연테이프
lower_line = np.array([0, 0, 0])
upper_line = np.array([360, 255, 150])

# 연습용, 실전용
#lower_line = np.array([70, 0, 0])
#upper_line = np.array([220, 70, 130])

roix_s = 300
roix_e = 400
roiy_s = 200
roiy_e = 480

In [4]:
def stop():
    tiki.stop()
    time.sleep(1)

def mission():
    tiki.set_motor_power("MOTOR_LEFT", 0)
    tiki.set_motor_power("MOTOR_RIGHT", 0)
    time.sleep(5)

def turn_90_degrees(direction):
    turn_duration = 0.75
    turn_power = 30  

    if direction == 'left':
        tiki.counter_clockwise(turn_power)
    elif direction == 'right':
        tiki.clockwise(turn_power)

    time.sleep(turn_duration)
    stop()

In [5]:
class Wrapper:
    def __init__(self, save_name=None):
        self.cap = cv2.VideoCapture(
            "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=640, height=480, framerate=15/1, format=NV12 ! "
            "nvvidconv flip-method=2 ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
        )
    def __enter__(self):
        print('entered')
        return self.cap
    def __exit__(self,a,b,c):
        self.cap.release()
        print('released')

In [6]:
def detect_lines(image):
    
    crop_img = image[roix_s:roix_e, roiy_s:roiy_e].copy()
    
    hsv_image = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)
    color_mask = cv2.inRange(hsv_image, lower_line, upper_line)
    masked_image = cv2.bitwise_and(crop_img, crop_img, mask=color_mask)
    gray_mask = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray_mask, 1, 255, cv2.THRESH_BINARY)
    lines, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return crop_img, lines

In [7]:
def adjust_wheel_speed(cx, cool_time):
    if cool_time != 0:
        tiki.set_motor_power(tiki.MOTOR_LEFT, 0)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 0)
    elif (100 < cx < 140): # NORMAL
        tiki.set_motor_power(tiki.MOTOR_LEFT, 20)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 20)   
    elif (cx > 140):
        if (cx > 220): # LEFT CORNER
            tiki.set_motor_power(tiki.MOTOR_LEFT, 30)
            tiki.set_motor_power(tiki.MOTOR_RIGHT, 10)
        else: # LINE
            tiki.set_motor_power(tiki.MOTOR_LEFT, 20)
            tiki.set_motor_power(tiki.MOTOR_RIGHT, 15)
    elif (cx < 100):
        if (cx < 40): # RIGHT CORNER
            tiki.set_motor_power(tiki.MOTOR_LEFT, 10)
            tiki.set_motor_power(tiki.MOTOR_RIGHT, 30)
        else: # LINE
            tiki.set_motor_power(tiki.MOTOR_LEFT, 15)
            tiki.set_motor_power(tiki.MOTOR_RIGHT, 20)
    
    return 

In [8]:
def detect_event_color(image, lower_event, upper_event):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_event, upper_event)
    return mask

def calculate_center_area(image, center_width=200, center_height=50):
    height, width = image.shape[:2]
    
    center_x = width // 2 
    center_y = height // 2 + 10
    start_x = center_x - center_width // 2
    start_y = center_y - center_height // 2
    end_x = start_x + center_width
    end_y = start_y + center_height

    return (start_x, start_y, end_x, end_y)

def detect_event_in_center(mask, center_coords):
    start_x, start_y, end_x, end_y = center_coords
    center_mask = mask[start_y:end_y, start_x:end_x]
    
    event_detected = np.sum(center_mask) > 0
    
    event_point = None
    if event_detected:
        contours, _ = cv2.findContours(center_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            M = cv2.moments(largest_contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"]) + start_x
                cy = int(M["m01"] / M["m00"]) + start_y
                event_point = (cx, cy)
    
    return event_detected, event_point

In [9]:
def Event_Handler(event_num):
    if event_num == 0:
        turn_90_degrees('left')
    elif event_num == 1: # mission
        turn_90_degrees('left')
        mission()
        turn_90_degrees('left')
    elif event_num == 2:
        turn_90_degrees('left')
    elif event_num == 3: # mission
        turn_90_degrees('left')
        mission()
        turn_90_degrees('right')
    elif event_num == 5:
        turn_90_degrees('right')
    elif event_num == 6: # mission
        turn_90_degrees('left')
        mission()
        turn_90_degrees('right')
    elif event_num == 7: # mission
        turn_90_degrees('left')
        mission()
        turn_90_degrees('right')
    elif event_num == 8:
        turn_90_degrees('left')
    elif event_num == 9:
        stop()
    

In [10]:
# 카메라에서 실시간 이미지 출력하는 함수
def main():
    video_widget = widgets.Image(format='jpeg')
    tiki.set_motor_mode(tiki.MOTOR_MODE_PID)
    event_detection_count = 0
    event_detection_threshold = 3
    
    event_cnt = 0
    
    cool_flag = 1
    cool_time = 0
    
    with Wrapper() as cap:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Line
            crop_img, lines = detect_lines(frame)
            
            if len(lines) > 0:
                
                c = max(lines, key=cv2.contourArea)
                M = cv2.moments(c)
                if M['m00'] != 0:
                    cx = int(M['m10']/M['m00'])
                    cy = int(M['m01']/M['m00'])

                #cv2.line(crop_img,(cx,0),(cx,640),(255,0,0),1)
                #cv2.line(crop_img,(0,cy),(480,cy),(255,0,0),1)
                cv2.drawContours(crop_img, lines, -1, (0,255,0), 1)

            adjust_wheel_speed(cx, cool_time)
            current_time = time.time()
            # Event Detection
            event_mask = detect_event_color(frame, lower_event, upper_event)
            center_coords = calculate_center_area(event_mask)
            event_detected, event_point = detect_event_in_center(event_mask, center_coords)
            
            if event_detected:
                event_detection_count += 1
            else:
                event_detection_count = 0 
                
            event_area = cv2.bitwise_and(frame, frame, mask=event_mask)
            
            alpha = 0.3
            mask = event_mask.astype(bool)
            frame[mask] = cv2.addWeighted(frame, alpha, event_area, 1 - alpha, 0)[mask]

            start_x, start_y, end_x, end_y = center_coords
            cv2.rectangle(frame, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)

            if event_point:
                cv2.circle(frame, event_point, 10, (0, 255, 0), -1)
                cv2.putText(frame, f"Green: ({event_point[0]}, {event_point[1]})", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            if cool_time <= 0:
                cool_flag = 1
                
            # Event Start 
            if(event_detection_count >= event_detection_threshold and cool_flag):
                stop()
                Event_Handler(event_cnt)
                event_cnt += 1
                cool_flag = 0
                cool_time = 100
            if cool_time > 0:
                cool_time -= 1
                
            
            frame[roix_s:roix_e, roiy_s:roiy_e] = crop_img

            cv2.putText(frame, f"Current Event: {event_cnt}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.putText(frame, f"cx: {cx}s", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

            cv2.putText(frame, f"Cool Time: {cool_time}s", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.putText(frame, f"Cool Flag: {cool_flag}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

            #left_encoder = tiki.get_encoder(tiki.MOTOR_LEFT)
            #right_encoder = tiki.get_encoder(tiki.MOTOR_RIGHT)
            #cv2.putText(frame, f"L_Motor: {left_encoder}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            #cv2.putText(frame, f"R_Motor: {right_encoders}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            
            _, buffer = cv2.imencode('.jpg', frame)
            video_widget.value = buffer.tobytes()
            
            clear_output(wait=True)
            display(video_widget)
            time.sleep(0.03)  # 프레임 간 간격

if __name__ == '__main__':
    main()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

released


KeyboardInterrupt: 